# Import packages

In [1]:
""" 
Inflation Decomposition Analysis
Last Updated: 2025-09-19
Author: Chris Kaczmarczyk-Smith, PhD
VP, Economy & Data ATMTA Inc.
"""

# Imports 

## Standard Fare
import numpy as np
import pandas as pd

## My preferred plotting package
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Simple linear model, statsmodels provides more analytic output than something like scipy
import statsmodels.api as sm

# Get Data and look through it

In [2]:
# Grabs data from public chris-economics repo
raw_base_url = "https://raw.githubusercontent.com/chris-economics/chris-economics/main/inflation_analysis"
money_url = f"{raw_base_url}/money_supply.csv"
cpi_url = f"{raw_base_url}/cpi.csv"
df_money = pd.read_csv(money_url)
df_cpi = pd.read_csv(cpi_url)

In [3]:
# Note that df_cpi is not aggregated to the day level. 
df_cpi.head()

,date,name,type,price_weighted
0,2025-08-30,Aerogel,compounds,4.852447e-11
1,2025-08-30,Aerogel Contract,contracts,9.532166e-09
2,2025-08-30,Ammunition,r4,1.922471e-08
3,2025-08-30,Armstrong IMP Tip,ships,4.197786e-07
4,2025-08-30,Biomass,r9,1.013195e-09


In [4]:
# Note that df_money starts 10 days prior to df_cpi.
df_money.head()

,date,atlas_inflation
0,2025-08-20,2335
1,2025-08-21,204940
2,2025-08-22,147021
3,2025-08-23,129616
4,2025-08-24,127184


# Calculate CPI from price data

In [5]:
# The bundle which we will select on
## There are several other types of items to explore
bundle={'r4':0}
df_cpi = df_cpi[df_cpi['type'].isin(bundle.keys())]

# Weighted AVG Price Per Day
dt_cpi = df_cpi.groupby(['date']).agg({'price_weighted':'mean','name':'nunique'}).reset_index()
dt_cpi.columns = ['date','price_weighted','num_goods']
# Generate Price Level - This automatically normalizes the data.
base = dt_cpi['price_weighted'].iloc[0]
dt_cpi['cpi'] = dt_cpi['price_weighted'] / base * 100

# Ensure that the number of goods in the bundle remain constant. 
fig = px.line(dt_cpi, x='date', y='num_goods', title='Unique Assets in Bundle Data')
fig.update_layout(showlegend=False,width=1000,height=400)
fig.show()

# Calculate Cumulative ATLAS and plot

In [6]:
# Get Money Supply
df_money['atlas_cumulative'] = df_money['atlas_inflation'].cumsum()

# Create a plotly figure for change+cumulative zATLAS
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(
        x=df_money['date'],
        y=df_money['atlas_inflation'],
        name=f'Daily Chhange',
        mode='lines'
    ),
    secondary_y=False
)
fig.add_trace(
    go.Scatter(
        x=df_money['date'],
        y=df_money['atlas_cumulative'],
        name='Cumulative',
        mode='lines',
        fill='tozeroy'
    ),
    secondary_y=True
)

print(f"Outstanding zATLAS: {df_money['atlas_cumulative'].iloc[-1]:,.2f}")
fig.update_layout(title='zATLAS Circulating Supply', width=1000, height=400)
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Daily Change', secondary_y=False)
fig.update_yaxes(title_text='Cumulative', secondary_y=True)
fig.show()

Outstanding zATLAS: 6,762,152.00


# Run the Model

In [7]:
# Final data set
# Merge Money Supply into price data
dt = dt_cpi.merge(df_money, on='date', how='left') # Important left merge, recall missing first 10 days in price data

# Normalize money supply between 0 and 1
dt['m1_norm'] = (
    dt['atlas_cumulative'] - dt['atlas_cumulative'].min()
    ) / (
    dt['atlas_cumulative'].max() - dt['atlas_cumulative'].min()
    )
dt.head()

,date,price_weighted,num_goods,cpi,atlas_inflation,atlas_cumulative,m1_norm
0,2025-08-30,1.656543e-08,4,100.000000,208090,1664214,0.000000
1,2025-08-31,1.603975e-08,4,96.826645,221959,1886173,0.043539
2,2025-09-01,1.268104e-08,4,76.551185,252226,2138399,0.093015
3,2025-09-02,1.625474e-08,4,98.124446,252536,2390935,0.142552
4,2025-09-03,1.256413e-08,4,75.845473,257610,2648545,0.193084


In [8]:
# Run regression: cpi ~ m1 + e
covariates = ['m1_norm']
outcome    = ['cpi']
X = sm.add_constant(dt[covariates])
model = sm.OLS(dt[outcome], X).fit()

# Display model
## high R-Squared (>0.8)
## Significant beta_1 (p = ~0.00)
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    cpi   R-squared:                       0.825
Model:                            OLS   Adj. R-squared:                  0.815
Method:                 Least Squares   F-statistic:                     89.37
Date:                Fri, 19 Sep 2025   Prob (F-statistic):           1.29e-08
Time:                        12:53:36   Log-Likelihood:                -124.35
No. Observations:                  21   AIC:                             252.7
Df Residuals:                      19   BIC:                             254.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -33.2581     39.769     -0.836      0.4

In [9]:
## First, find the residual from the regression - this is the component we care about
dt['cpi_adj'] = model.resid 
## Second, add the regression intercept (b_0) - this rescales the number to start at 100.
dt['cpi_adj'] = dt['cpi_adj']+model.params['const']

# Get Adjusted and Unadjusted Inflation
pi_bar = dt['cpi'].iloc[-1]
pi_bar_adj = dt['cpi_adj'].iloc[-1]

# Display output
print(f"Latest Unadjusted CPI: {pi_bar:,.2f}")
print(f"Latest Adjusted CPI:   {pi_bar_adj:,.2f}")
fig = px.line(dt, x='date', y=['cpi','cpi_adj'], title='Price Level and Price Level Adjusted for M1')
fig.update_layout(showlegend=True,width=1000,height=400)
fig.show()

Latest Unadjusted CPI: 690.41
Latest Adjusted CPI:   57.31
